In [1]:
import pandas as pd
from splinter import Browser
import time
import pymongo

In [2]:
url = 'https://www.espn.com/nfl/stats/player/_/stat/rushing'

In [3]:
executable_path = {'executable_path': 'C:/Users/Daniel/Desktop/Drivers/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
browser.visit(url)
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    rushing_tables= pd.read_html(browser.html)

cycle complete


In [5]:
df=rushing_tables[0]
df_2=rushing_tables[1]

In [20]:
# teams_list=[]
# names_list=[]
# for item in df.Name:
#     if item[-3].isupper():
#         teams_list.append(item[-3:])
#         names_list.append(item[:-3])
#     else:
#         teams_list.append(item[-2:])
#         names_list.append(item[:-2])
# df.Name=names_list
# df['Team Name']=teams_list

##look into camelcase to help with stripping issue
##look into regex library

In [6]:
mf=df.join(df_2)


mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)

In [7]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db=client.NFL_StatsDB
collection=db.rushing
data1 = mf.to_dict(orient='records')
collection.insert_many(data1)